In [1]:
!pip install -qq git+https://www.github.com/keras-team/keras-contrib.git

In [2]:
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import os
import numpy as np
from skimage import io, exposure
from skimage.filters import laplace
from skimage.transform import resize
import cv2

#create a new output folder
#os.mkdir('/kaggle/working/CXR_Mask/')

# define location of dataset for Main lung CXR images
folder1 = '../input/jsrt-data/'
folder2 = '../input/montgomery-data-npy-format/'

#Load all the chest Xrays from respective datasets
jsrt_CXR = np.load(folder1 + 'CXR.npy')
montgomery_CXR = np.load(folder2 + 'CXR.npy')

print(jsrt_CXR.shape,  montgomery_CXR.shape)

(246, 256, 256, 1) (138, 256, 256, 1)


In [3]:
from numpy import savez_compressed

# Split into train and test data(67%)
X=np.arange(0,246)
X_train, X_test = train_test_split(X, test_size=0.67, random_state=999)
jsrt_CXR_train = jsrt_CXR[X_train]
jsrt_CXR_test = jsrt_CXR[X_test]

X=np.arange(0,138)
X_train, X_test = train_test_split(X, test_size=0.4, random_state=999)
montgomery_CXR_train = montgomery_CXR[X_train]
montgomery_CXR_test = montgomery_CXR[X_test]

print(jsrt_CXR_train.shape, montgomery_CXR_train.shape)
#combine and save datasets together
savez_compressed('cyclegan_CXR_train.npz', jsrt_CXR_train, montgomery_CXR_train)

save('jsrt_CXR_train.npy', jsrt_CXR_train)
save('montgomery_CXR_train.npy', montgomery_CXR_train)
save('jsrt_CXR_test.npy',jsrt_CXR_test)
save('montgomery_CXR_test.npy', montgomery_CXR_test)

(81, 256, 256, 1) (82, 256, 256, 1)


In [4]:
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot

# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 * 2.0) - 1.0
	X2 = (X2 * 2.0) - 1.0
	return [X1, X2]

# define the discriminator model
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_image = Input(shape=image_shape)
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
	return model

# Applying UNET for generator======================================
from keras.models import Model
from keras.layers.merge import concatenate
from keras.layers import Input, Convolution2D, MaxPooling2D, UpSampling2D

# define the standalone generator model
def define_generator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	#==============UNET STARTS============================================
	k_size = 3
	merge_axis = -1 # Feature maps are concatenated along last axis (for tf backend)
	conv1 = Convolution2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(g)
	conv1 = Convolution2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(conv1)
	pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
	u1 = InstanceNormalization(axis=-1)(pool1)
    
	conv2 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(u1)
	conv2 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv2)
	pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
	u2 = InstanceNormalization(axis=-1)(pool2)

	conv3 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(u2)
	conv3 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv3)
	pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
	u3 = InstanceNormalization(axis=-1)(pool3)

	conv4 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(u3)
	conv4 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv4)
	pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
	u4 = InstanceNormalization(axis=-1)(pool4)

	conv5 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(u4)

	up1 = UpSampling2D(size=(2, 2))(conv5)
	conv6 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up1)
	conv6 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv6)
	u6 = InstanceNormalization(axis=-1)(conv6)
	merged1 = concatenate([conv4, u6], axis=merge_axis)
	conv6 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged1)

	up2 = UpSampling2D(size=(2, 2))(conv6)
	conv7 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up2)
	conv7 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv7)
	u7 = InstanceNormalization(axis=-1)(conv7)
	merged2 = concatenate([conv3, u7], axis=merge_axis)
	conv7 = Convolution2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged2)

	up3 = UpSampling2D(size=(2, 2))(conv7)
	conv8 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(up3)
	conv8 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv8)
	u8 = InstanceNormalization(axis=-1)(conv8)
	merged3 = concatenate([conv2, u8], axis=merge_axis)
	conv8 = Convolution2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(merged3)

	up4 = UpSampling2D(size=(2, 2))(conv8)
	conv9 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(up4)
	conv9 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv9)
	u9 = InstanceNormalization(axis=-1)(conv9)
	merged4 = concatenate([conv1, u9], axis=merge_axis)
	conv9 = Convolution2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(merged4)

	conv10 = Convolution2D(filters=1, kernel_size=k_size, padding='same', activation='sigmoid')(conv9)
	#==================================UNET Ends=====================
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(conv10)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
    #g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	g = Conv2D(1, (7,7), padding='same', kernel_initializer=init)(g)   # 1 channel for grayscale
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	output_id = g_model_1(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	return model

# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]    
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
	# save the first generator model
	filename1 = 'g_model_AtoB.h5'
	g_model_AtoB.save(filename1)
	# save the second generator model
	filename2 = 'g_model_BtoA.h5'
	g_model_BtoA.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
	step = step/trainX.shape[0]
	# select a sample of input images
	X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate translated images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# scale all pixels from [-1,1] to [0,1]
	X_in = (X_in + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	X_in = np.squeeze(X_in)    #convert to one channel (3D to 1D)
	X_out = np.squeeze(X_out)  #convert to one channel (3D to 1D)
	# plot real images
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_in[i], cmap='gray')
	# plot translated image
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + n_samples + i)
		pyplot.axis('off')        
		pyplot.imshow(X_out[i], cmap='gray')        
	# save plot to file
	filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
	pyplot.savefig(filename1)
	pyplot.close()

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)

# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
	# define properties of the training run
	n_epochs, n_batch, = 200, 1
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
		if (i+1) % (bat_per_epo * 5) == 0:
			# save the models
			save_models(i, g_model_AtoB, g_model_BtoA)

In [5]:
from keras.utils.vis_utils import model_to_dot
import os
from IPython.display import Image

#Image(model_to_dot(g_model_AtoB, show_shapes=True).create_png())

# load image data
dataset = load_real_samples('cyclegan_CXR_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

Loaded (81, 256, 256, 1) (82, 256, 256, 1)
>1, dA[1.354,0.703] dB[1.503,0.872] g[20.924,20.420]
>2, dA[8.150,1.196] dB[11.376,1.451] g[19.917,19.304]
>3, dA[6.574,1.295] dB[5.345,0.646] g[19.229,19.355]
>4, dA[6.529,0.597] dB[8.264,0.551] g[18.978,19.009]
>5, dA[3.581,0.700] dB[6.231,0.626] g[19.692,19.682]
>6, dA[5.388,0.496] dB[2.632,0.581] g[18.839,18.857]
>7, dA[2.129,0.586] dB[0.995,0.572] g[19.514,18.769]
>8, dA[1.354,0.447] dB[1.145,0.510] g[19.692,18.949]
>9, dA[1.147,0.497] dB[1.009,0.472] g[19.170,19.283]
>10, dA[0.665,0.503] dB[2.061,0.516] g[19.616,18.977]
>11, dA[0.699,0.447] dB[4.354,0.488] g[19.645,19.203]
>12, dA[0.768,0.373] dB[2.205,0.453] g[18.398,17.982]
>13, dA[0.924,0.391] dB[0.928,0.474] g[19.537,19.288]
>14, dA[0.576,0.355] dB[1.622,0.493] g[19.044,18.543]
>15, dA[0.344,0.373] dB[0.351,0.421] g[18.175,17.850]
>16, dA[0.434,0.391] dB[0.433,0.425] g[17.812,17.576]
>17, dA[0.466,0.318] dB[0.535,0.411] g[18.835,18.597]
>18, dA[0.325,0.367] dB[0.615,0.390] g[18.323,1